In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

In [2]:
city_weather_df = pd.read_csv("../WeatherPy/cities.csv")
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Fălciu,46.2961,28.1408,68.65,92,12,2.89,RO,1628124561
1,Lavrentiya,65.5833,-171.0000,60.71,72,12,20.33,RU,1628124562
2,Punta Arenas,-53.1500,-70.9167,46.51,71,0,11.50,CL,1628124562
3,Lakeway,30.3638,-97.9796,88.25,65,1,3.00,US,1628124563
4,Pinerovka,51.5620,43.0689,62.22,68,19,3.51,RU,1628124564
...,...,...,...,...,...,...,...,...,...
562,Kaupanger,61.1845,7.2404,53.26,81,25,2.15,NO,1628124937
563,Tsiroanomandidy,-18.7667,46.0333,57.00,83,0,1.41,MG,1628124938
564,Wąbrzeźno,53.2799,18.9477,58.10,85,98,6.20,PL,1628124939
565,Camacha,33.0833,-16.3333,70.56,83,75,16.11,PT,1628124818


In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations
locations = city_weather_df[["Lat","Lng"]]

#Add Heatmap layer to map
humidity = city_weather_df["Humidity"]

In [4]:
#Plot heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#Narrow down the cities to fit weather conditions
#Drop any rows will null values
smaller_city_df = city_weather_df.loc[(city_weather_df["Wind Speed"] <= 10) & (city_weather_df["Clouds"]== 0) & \
                  (city_weather_df["Max Temp"] >= 70) & (city_weather_df["Max Temp"] <= 80)].dropna()
smaller_city_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
77,Ribeira Grande,38.5167,-28.7000,72.75,89,0,6.22,PT,1628124613
171,West Helena,34.5507,-90.6418,77.95,63,0,6.91,US,1628124663
204,Ariquemes,-9.9133,-63.0408,75.27,42,0,2.15,BR,1628124698
250,Aripuanã,-9.1667,-60.6333,74.34,44,0,2.30,BR,1628124729
259,Marsá Maţrūḩ,31.3525,27.2453,79.00,89,0,6.91,EG,1628124735
264,Bandar-e Anzali,37.4727,49.4622,78.15,61,0,4.32,IR,1628124739
377,Macamic,48.7502,-78.9996,73.63,77,0,7.47,CA,1628124814
495,Mitsamiouli,-11.3847,43.2844,74.25,73,0,1.25,KM,1628124892


In [7]:
#Store into variable named hotel_df
hotel_df = smaller_city_df.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
77,Ribeira Grande,PT,38.5167,-28.7000,
171,West Helena,US,34.5507,-90.6418,
204,Ariquemes,BR,-9.9133,-63.0408,
250,Aripuanã,BR,-9.1667,-60.6333,
259,Marsá Maţrūḩ,EG,31.3525,27.2453,
264,Bandar-e Anzali,IR,37.4727,49.4622,
377,Macamic,CA,48.7502,-78.9996,
495,Mitsamiouli,KM,-11.3847,43.2844,


In [8]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [11]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    #Due to recent events, when I am to actually google "closest hotel in...
    #I get a COVID-19 alert and a link to the location's travel advisory page
    #This naturally makes almost every result an error
    except (KeyError, IndexError):
        print("None round")
        
    print("--------------------End of Search")

Retrieving Results for Index 77: Ribeira Grande.
Closest hotel in Ribeira Grande is BELO CAMPO.
--------------------End of Search
Retrieving Results for Index 171: West Helena.
Closest hotel in West Helena is Best Western Inn.
--------------------End of Search
Retrieving Results for Index 204: Ariquemes.
Closest hotel in Ariquemes is Premium Palace Hotel.
--------------------End of Search
Retrieving Results for Index 250: Aripuanã.
None round
--------------------End of Search
Retrieving Results for Index 259: Marsá Maţrūḩ.
Closest hotel in Marsá Maţrūḩ is Arous El Bahr Hotel.
--------------------End of Search
Retrieving Results for Index 264: Bandar-e Anzali.
Closest hotel in Bandar-e Anzali is Grand Hotel.
--------------------End of Search
Retrieving Results for Index 377: Macamic.
Closest hotel in Macamic is Maison Tardif Enr. - Auberge à Macamic, QC.
--------------------End of Search
Retrieving Results for Index 495: Mitsamiouli.
Closest hotel in Mitsamiouli is Al Camar Lodge.
-----

In [12]:
 #Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
77,Ribeira Grande,PT,38.5167,-28.7000,BELO CAMPO
171,West Helena,US,34.5507,-90.6418,Best Western Inn
204,Ariquemes,BR,-9.9133,-63.0408,Premium Palace Hotel
250,Aripuanã,BR,-9.1667,-60.6333,
259,Marsá Maţrūḩ,EG,31.3525,27.2453,Arous El Bahr Hotel
264,Bandar-e Anzali,IR,37.4727,49.4622,Grand Hotel
377,Macamic,CA,48.7502,-78.9996,"Maison Tardif Enr. - Auberge à Macamic, QC"
495,Mitsamiouli,KM,-11.3847,43.2844,Al Camar Lodge


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Plot markers on top of the heatmap
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))